In [11]:
import pandas as pd

df = pd.read_csv('data/근로시간.csv')
# df.head()

,시점,서울,광역시,대도시외
0,2023,154.6,1077.5,1384.4
1,2022,160.2,1116.8,1429.8
2,2021,167.0,1172.2,1497.0
3,2020,151.6,1075.9,1379.1
4,2019,171.5,1199.1,1359.9


In [12]:
df_melted = df.melt(id_vars=['시점'], var_name='지역별', value_name='근로시간')
df_melted.head(),df_melted['지역별'].unique()

(     시점 지역별   근로시간
 0  2023  서울  154.6
 1  2022  서울  160.2
 2  2021  서울  167.0
 3  2020  서울  151.6
 4  2019  서울  171.5,
 array(['서울', '광역시', '대도시외'], dtype=object))

In [13]:
# '시점'을 datetime 형식으로 변환
df['시점'] = pd.to_datetime(df['시점'].astype(str) + '-01-01')

# 인덱스로 설정
yearly_data = df.set_index('시점')

# 지역별로 그룹화 후 리샘플링
monthly_dfs = []

for region, group in yearly_data.groupby('지역별'):
    monthly_df = group.resample('ME').agg({'근로시간': 'mean'})
    
    # 보간
    monthly_df = monthly_df.interpolate(method='linear')
    
    # NaN 값 처리
    monthly_df.fillna(0, inplace=True)
    
    # 시점 다시 설정
    monthly_df.reset_index(inplace=True)
    
    # '시점'이 datetime인지 확인 후, 형식 변경
    if pd.api.types.is_datetime64_any_dtype(monthly_df['시점']):
        monthly_df['시점'] = monthly_df['시점'].dt.strftime('%Y.%m')
    
    # 지역별 컬럼 추가
    monthly_df['지역별'] = region
    
    monthly_dfs.append(monthly_df)

# 모든 지역별 데이터를 하나의 DataFrame으로 결합
final_monthly_df = pd.concat(monthly_dfs, ignore_index=True)

final_monthly_df.head(12), final_monthly_df.shape, final_monthly_df['지역별'].unique()

# # 데이터 저장
# monthly_df.to_csv('시도별사교육참여율(월별).csv', index=False)

KeyError: '지역별'